In [1]:
#Импорты
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
#Загрузка изображений из папки
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [3]:
#Получение данных определённого размера для тренировки
def create_training_data(target_size):
    images = []
    labels = []
    persons = os.listdir('data')
    num_classes = len(persons)

    for person_index, person in enumerate(persons):
        folder_path = os.path.join('data', person)
        person_images = load_images_from_folder(folder_path)
        resized_images = [cv2.resize(img, target_size) for img in person_images]
        images.extend(resized_images)
        labels.extend([person_index] * len(resized_images))

    images = np.array(images)
    labels = np.array(labels)
    
    return images, labels, num_classes

In [4]:
def preprocess_data(data_pairs):
    preprocessed_pairs = []
    for pair in data_pairs:
        preprocessed_pair = []
        for image in pair:
            image_array = np.array(image)
            image_array = image_array / 255.0
            preprocessed_pair.append(image_array)
        preprocessed_pairs.append(preprocessed_pair)
    return np.array(preprocessed_pairs)


In [5]:
#Сама модель нейронки, на вход 150х150х3 картинка, на выходе определяет принодлежность к классу из num_classes
def create_model():
    input_shape = (150, 150, 3)
    
    # Модель для первого изображения
    input1 = Input(shape=input_shape)
    x1 = Conv2D(64, (3, 3), activation='relu')(input1)
    x1 = MaxPooling2D(pool_size=(2, 2))(x1)
    x1 = Conv2D(128, (3, 3), activation='relu')(x1)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D(pool_size=(2, 2))(x1)
    x1 = Flatten()(x1)
    x1 = Dense(128, activation='relu')(x1)
    
    # Модель для второго изображения
    input2 = Input(shape=input_shape)
    x2 = Conv2D(64, (3, 3), activation='relu')(input2)
    x2 = MaxPooling2D(pool_size=(2, 2))(x2)
    x2 = Conv2D(128, (3, 3), activation='relu')(x2)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D(pool_size=(2, 2))(x2)
    x2 = Flatten()(x2)
    x2 = Dense(128, activation='relu')(x2)
    
    # Объединение двух ветвей
    merged = tf.keras.layers.concatenate([x1, x2])
    
    x = Dense(128, activation='relu')(merged)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[input1, input2], outputs=output)
    
    return model

In [6]:
target_size = (150, 150)
images, labels, num_classes = create_training_data(target_size)

image_pairs = []
label_pairs = []

count=0
for i in range(len(images)):
    for j in range(i , len(images)):
        image_pairs.append([images[i], images[j]])
        if labels[i] == labels[j]:
            label_pairs.append(True)
        else:
            label_pairs.append(False)

image_pairs = np.array(image_pairs)
label_pairs = np.array(label_pairs)

In [7]:
x_train_pairs, x_val_pairs, y_train_pairs, y_val_pairs = train_test_split(image_pairs, label_pairs, test_size=0.2, random_state=42)

y_train_pairs = np.array(y_train_pairs).astype(int)
y_val_pairs = np.array(y_val_pairs).astype(int)

x_train_pairs = preprocess_data(x_train_pairs)
x_val_pairs = preprocess_data(x_val_pairs)

In [8]:
learning_rate = 0.001
batch_size = 64
epochs = 30

optimizer = Adam(learning_rate=learning_rate)

In [9]:
#обучение
"""
best_val_accuracy = 0.0

for run in range(20):
    print(f"Запуск {run}")
    model = create_model()
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'], run_eagerly=True)
    checkpoint = ModelCheckpoint('retrained.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    history = model.fit(
        [x_train_pairs[:, 0], x_train_pairs[:, 1]],
        y_train_pairs,
        batch_size=batch_size,
        epochs=epochs,  
        shuffle=True,
        validation_data=([x_val_pairs[:, 0], x_val_pairs[:, 1]], y_val_pairs),
        callbacks=[checkpoint]
    )
    
    val_accuracy = max(history.history['val_accuracy'])
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        model.load_weights('retrained.h5')
        model.save('best_retrained.h5')
    
    print("Текущая точность на валидации:", val_accuracy)
    print("Лучшая точность на валидации:", best_val_accuracy)
"""

'\nbest_val_accuracy = 0.0\n\nfor run in range(20):\n    print(f"Запуск {run}")\n    model = create_model()\n    model.compile(loss=\'binary_crossentropy\', optimizer=optimizer, metrics=[\'accuracy\'], run_eagerly=True)\n    checkpoint = ModelCheckpoint(\'retrained.h5\', monitor=\'val_accuracy\', verbose=1, save_best_only=True, mode=\'max\')\n    \n    history = model.fit(\n        [x_train_pairs[:, 0], x_train_pairs[:, 1]],\n        y_train_pairs,\n        batch_size=batch_size,\n        epochs=epochs,  \n        shuffle=True,\n        validation_data=([x_val_pairs[:, 0], x_val_pairs[:, 1]], y_val_pairs),\n        callbacks=[checkpoint]\n    )\n    \n    val_accuracy = max(history.history[\'val_accuracy\'])\n    if val_accuracy > best_val_accuracy:\n        best_val_accuracy = val_accuracy\n        model.load_weights(\'retrained.h5\')\n        model.save(\'best_retrained.h5\')\n    \n    print("Текущая точность на валидации:", val_accuracy)\n    print("Лучшая точность на валидации:", 

In [10]:
model = create_model()
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'], run_eagerly=True)
model.load_weights('model/best_retrained.h5')

epochs=131

best_val_accuracy = model.evaluate([x_val_pairs[:, 0], x_val_pairs[:, 1]], y_val_pairs)[1]
print('Текущий accuracy: ', best_val_accuracy)

32/32 [==============================] - 19s 604ms/step - loss: 0.0841 - accuracy: 0.9931
Текущий accuracy:  0.9930692911148071


In [11]:
"""while True:
    checkpoint = ModelCheckpoint('retrained.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    history = model.fit(
        [x_train_pairs[:, 0], x_train_pairs[:, 1]],
        y_train_pairs,
        batch_size=batch_size,
        epochs=epochs + 10,
        shuffle=True,
        validation_data=([x_val_pairs[:, 0], x_val_pairs[:, 1]], y_val_pairs),
        callbacks=[checkpoint],
        initial_epoch=epochs
    )

    val_accuracy = max(history.history['val_accuracy'])
    print("Точность на валидации после новой эпох:", val_accuracy)

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        model.load_weights('retrained.h5')
        model.save_weights('model/best_retrained.h5')
        epochs += 10
        print("Новые веса сохранены")

    else:
        model.load_weights('model/best_retrained.h5')
        print("Загружены предыдущие лучшие веса")
        
    if val_accuracy >= 0.999:
        break"""

'while True:\n    checkpoint = ModelCheckpoint(\'retrained.h5\', monitor=\'val_accuracy\', verbose=1, save_best_only=True, mode=\'max\')\n    \n    history = model.fit(\n        [x_train_pairs[:, 0], x_train_pairs[:, 1]],\n        y_train_pairs,\n        batch_size=batch_size,\n        epochs=epochs + 10,\n        shuffle=True,\n        validation_data=([x_val_pairs[:, 0], x_val_pairs[:, 1]], y_val_pairs),\n        callbacks=[checkpoint],\n        initial_epoch=epochs\n    )\n\n    val_accuracy = max(history.history[\'val_accuracy\'])\n    print("Точность на валидации после новой эпох:", val_accuracy)\n\n    if val_accuracy > best_val_accuracy:\n        best_val_accuracy = val_accuracy\n        model.load_weights(\'retrained.h5\')\n        model.save_weights(\'model/best_retrained.h5\')\n        epochs += 10\n        print("Новые веса сохранены")\n\n    else:\n        model.load_weights(\'model/best_retrained.h5\')\n        print("Загружены предыдущие лучшие веса")\n        \n    if va

In [12]:
model.save('model/model_v0.993069.h5')

In [13]:
def predict_pair(image_path1, image_path2, model):
    image1 = load_and_preprocess_image(image_path1)
    image2 = load_and_preprocess_image(image_path2)
    prediction = model.predict([[image1], [image2]])
    if prediction > 0.5:
        print("Да")
    else:
        print("Нет")

def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (150, 150))
    image = img_to_array(image)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

In [14]:
model = create_model()
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'], run_eagerly=True)
model.load_weights('model/best_retrained.h5')

predict_pair('test/test_ (1).jpg','test/test_ (1).jpg', model)#Да
predict_pair('test/test_ (1).jpg','test/test_ (2).jpg', model)#Да
predict_pair('test/test_ (1).jpg','test/test_ (3).jpg', model)#Нет
predict_pair('test/test_ (1).jpg','test/test_ (4).jpg', model)#Нет
predict_pair('test/test_ (1).jpg','test/test_ (5).jpg', model)#Нет
predict_pair('test/test_ (1).jpg','test/test_ (6).jpg', model)#Нет

1/1 [==============================] - 0s 53ms/step
Да
1/1 [==============================] - 0s 59ms/step
Да
1/1 [==============================] - 0s 57ms/step
Нет
1/1 [==============================] - 0s 54ms/step
Нет
1/1 [==============================] - 0s 68ms/step
Нет
1/1 [==============================] - 0s 54ms/step
Нет


In [15]:
predict_pair('test/test_ (2).jpg','test/test_ (2).jpg', model)#Да
predict_pair('test/test_ (2).jpg','test/test_ (3).jpg', model)#Нет
predict_pair('test/test_ (2).jpg','test/test_ (4).jpg', model)#Нет
predict_pair('test/test_ (2).jpg','test/test_ (5).jpg', model)#Нет
predict_pair('test/test_ (2).jpg','test/test_ (6).jpg', model)#Нет

1/1 [==============================] - 0s 55ms/step
Да
1/1 [==============================] - 0s 62ms/step
Нет
1/1 [==============================] - 0s 59ms/step
Нет
1/1 [==============================] - 0s 53ms/step
Нет
1/1 [==============================] - 0s 57ms/step
Нет


In [16]:
predict_pair('test/test_ (3).jpg','test/test_ (3).jpg', model)#Да
predict_pair('test/test_ (3).jpg','test/test_ (4).jpg', model)#Нет
predict_pair('test/test_ (3).jpg','test/test_ (5).jpg', model)#Нет
predict_pair('test/test_ (3).jpg','test/test_ (6).jpg', model)#Нет

1/1 [==============================] - 0s 54ms/step
Да
1/1 [==============================] - 0s 58ms/step
Нет
1/1 [==============================] - 0s 63ms/step
Нет
1/1 [==============================] - 0s 60ms/step
Да


In [17]:
predict_pair('test/test_ (4).jpg','test/test_ (4).jpg', model)#Да
predict_pair('test/test_ (4).jpg','test/test_ (5).jpg', model)#Нет
predict_pair('test/test_ (4).jpg','test/test_ (6).jpg', model)#Нет
predict_pair('test/test_ (5).jpg','test/test_ (5).jpg', model)#Да
predict_pair('test/test_ (5).jpg','test/test_ (6).jpg', model)#Да
predict_pair('test/test_ (6).jpg','test/test_ (6).jpg', model)#Да

1/1 [==============================] - 0s 54ms/step
Да
1/1 [==============================] - 0s 57ms/step
Нет
1/1 [==============================] - 0s 55ms/step
Нет
1/1 [==============================] - 0s 56ms/step
Нет
1/1 [==============================] - 0s 56ms/step
Нет
1/1 [==============================] - 0s 69ms/step
Да
